# chroma_luma
This notebook explores the various chroma (color) and luma (brightness) features that can be extracted from individual frames. We'll be making extensive use of the `OpenCV` computer vision library. Functionality demonstrated here will be replicated in a functions file, and then used to populate the frame-level DataFrame.

In [1]:
import cv2
import os
from vision_features_io import *

In [2]:
film = 'parasite'
frame = 933
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'

### Loading an image
We can load the image and get its width and height.

In [3]:
# this frame is 854 (width) x 358 (height) pixels and has a BGR value
image = cv2.imread(img_path)
image.shape

(358, 854, 3)

We can specify a (width, height) coordinate and get the BGR value, consisting of three color channels.

In [4]:
image[10][20]

array([53, 74, 71], dtype=uint8)

# Blank Frames
The easiest frame to spot is the blank frame, with either all black or all white pixels. These could be used to identify scene transitions, such as when a scene ends with a dip-to-black or dip-to-white, transitioning to blank frames before the next scene.

### All-black frames
We can calculate the mean luminosity of the image by calculating the average of every BGR value for every pixel.

In [5]:
film = 'parasite'
frame = 22
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'
image = cv2.imread(img_path)

In [6]:
# every pixel in this frame has a BGR value of (0, 0, 0)
image.mean()

0.0

In [7]:
if image.mean() < 3: # threshold of 3, to be safe
    print('black frame detected')

black frame detected


### All-white frames

In [8]:
if image.mean() > 252:
    print('white frame detected')

# Luma
Measurements relating to luma, or single-scale (black-and-white) brightness.
### Mean brightness
We can take the brightness of the frame by calculating the average brightness of each pixel. But because each pixel is a color BGR element, we must first convert the image to grayscale, because brightness doesn't map linearly to BGR values. After converting each pixels' three BGR values to a single grayscale value, we can take the mean of the pixels' grayscale values.

In [9]:
film = 'booksmart'
frame = 4102
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'
image = cv2.imread(img_path)

In [10]:
# this is NOT the proper way to calculate brightness
image.mean()

114.32336262926611

In [11]:
# this conversion to grayscale is necessary
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray.mean()

111.82475067757808

### Maximum brightness
Although black values always sit at the end of the spectrum (0, 0, 0), white values in H.264-compressed videos rarely approach the pure white point of (255, 255, 255) because of gamma correction, a bandwidth-saving technique. On a display, the human eye doesn't need to see pure white to interpret pure white. Brightness is a power-scale, not a linear-scale, and so gamma correction can be used to reduce the amount of data used to convey white.

Below, we can search for the frame's brightest pixel. We may use this to scale white values. Remember that when searching for luma intensity, we need to convert to grayscale first.

In [12]:
brightest_pixel = 0
brightest_coordinate = 0
x = 0

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

for a in gray:
    y = 0
    for b in a:
        if b > brightest_pixel:
            brightest_pixel = b
            brightest_coordinate = (x, y)
        y += 1
    x += 1
print(brightest_pixel)
print(brightest_coordinate)

207
(4, 412)


### Removing Highlights and Shadows
Frames may contain lots of highlights and shadows, pixels that appear all-white or all-black. We may want to disregard these, as they don't contain useful color information.

One side effect of discarding these pixels is losing the original `shape` of the frame information. Instead of the (width, height, RGB) shape, we're left with an unrowed (pixel, RGB) array.

First, we'll unrow the frame array, and then search for the brightest and darkest pixels. We'll discard pixels that are close in intensity to those brightest and darkest, and we're left with just pixels with good color information.

In [13]:
frame = load_frame('booksmart', 3744)
brightest = brightest_pixel_intensity(frame)
darkest = darkest_pixel_intensity(frame)
unrowed = unrow_frame(frame)

if brightest >= 200:

    highlights_removed_list = []
    for pixel in unrowed:
        if pixel.mean() < brightest * .90:
            highlights_removed_list.append(pixel)

    unrowed = np.array(highlights_removed_list)
    
if darkest <= 15:
    shadows_removed_list = []
    for pixel in unrowed:
        if pixel.mean() > darkest + 10:
            shadows_removed_list.append(pixel)

    unrowed = np.array(shadows_removed_list)

print(unrowed.shape)

(281230, 3)


# Chroma
Measurements related to chroma, the BGR color information of the image.

### Extracting BGR
Each pixel has three additive values that make up its intensity and color: one 0-255 value each for Blue, Green, and Red. Although `OpenCV` has functions that can automatically extract things like the frame's mean value for each color channel, this won't work if we've unrowed the image data (such as in the previous code-block).

In [14]:
b = []
g = []
r = []

for pixel in unrowed:
    b.append(pixel[0])
    g.append(pixel[1])
    r.append(pixel[2])

b = np.array(b)
g = np.array(g)
r = np.array(r)

print(b.shape)
print(g.shape)
print(r.shape)

(281230,)
(281230,)
(281230,)


### Dominant Color
Generally, most frames have relatively balanced BGR color values. But some frames may have a dominant color, either as an artistic choice (leaning heavily into blue to make a scene feel "cooler"), or because of the actual events happening in the scene (an underwater scene).

We can determine if frames have predominantly blue, green, or red colors by looking at the intensity of each color as a proportion of the frame's overall intensity. So if any of the three intensities is more than 50% of the total intensity of the image, we declare that the dominant color.

BGR (RGB) is an additive color space, and secondary colors are created by mixing two of the primary colors together. So if there was a high intensity of blue and red (which makes magenta), there would be a low intensity of green. If the image has a low proportion of green, less than 10%, we would declare the dominant color to be magenta.

In [15]:
frame = load_frame('booksmart', 4072)
mid_pixels = remove_highlights_shadows(frame)
b, g, r = bgr(mid_pixels)

In [16]:
primary_threshold = .5
secondary_threshold = .1
if frame.mean() < 30:
    print('too dark to measure color')
elif b.mean() / (mid_pixels.mean() * 3) > primary_threshold:
    print('blue')
elif g.mean() / (mid_pixels.mean() * 3) > primary_threshold:
    print('green')
elif r.mean() / (mid_pixels.mean() * 3) > primary_threshold:
    print('red')
elif b.mean() / (mid_pixels.mean() * 3) < secondary_threshold:
    print('yellow')
elif g.mean() / (mid_pixels.mean() * 3) < secondary_threshold:
    print('magenta')
elif r.mean() / (mid_pixels.mean() * 3) < secondary_threshold:
    print('cyan')
else:
    print('no dominant color')

cyan
